In [1]:
from qtensor import Info, MPS #, Gates, CircuitCXFid
import torch
import numpy as np
import matplotlib.pyplot as plt
import time

N = 5

info = Info()

# Тест для генерации E

In [2]:
from qtensor import MPO

mps = MPS(info)
mps.get_random_mps(N, 1)
mpo = MPO(info)
mpo.pure_state(mps)

### проверка совпадение

In [3]:

indices = []
def gen(pref):
    if len(pref) == N:
        global indices
        indices.append(pref)
        return
    for i in range(2):
        gen(pref + [i])
        
gen([])
for i1 in indices:
    for i2 in indices:
        val1 = mps.get_element(i1)
        val2 = mps.get_element(i2)
        corr = torch.conj(val1) * val2
        our = mpo.get_element(i1, i2)
        diff = torch.abs(corr - our)
        assert diff.item() < 1e-12

### проверка trace

In [4]:
mpo.get_trace()

tensor(1.+0.j, dtype=torch.complex128)

# Произведение

In [5]:

def gen_E():
    info = Info()
    mps = MPS(info)
    
    mps.get_random_mps(N, 2)

    mpo = MPO(info)
    mpo.all_zeros_state(N)
    mpo.pure_state(mps)
    return mpo

mpo1 = gen_E()
mpo2 = gen_E()
A1 = mpo1.get_full_matrix()
A2 = mpo2.get_full_matrix()
B = mpo1.get_product(mpo2).get_full_matrix()
assert torch.sum(torch.abs((A1 @ A2) - B)) < 1e-12

# Генерация случайной MPO

In [6]:
mpo = MPO(info)

mpo.random_full(N, 2)

In [7]:
mpo.get_trace()

tensor(254.2428+13.5278j, dtype=torch.complex128)

In [8]:
mpo.r

[1, 2, 2, 2, 2, 1]

In [9]:
A = mpo.get_full_matrix()
print(A.trace())

tensor(254.2428+13.5278j, dtype=torch.complex128)


In [10]:
mpo_T = mpo.transpose()
AT = mpo_T.get_full_matrix()

assert torch.sum(torch.abs(A.T - AT)) < 1e-12

# Генерация случайной положительно определенной rho

In [11]:
mpo = MPO(info)
mpo.random_rho(N, 2)

In [12]:
mpo.get_trace()

tensor(1.0000-5.1082e-18j, dtype=torch.complex128)

In [13]:
mpo.r

[1, 16, 16, 16, 16, 1]

In [14]:
mpo1 = MPO(info)
mpo1.random_rho(N, 2)
mpo2 = MPO(info)
mpo2.random_rho(N, 2)
A1 = mpo1.get_full_matrix()
A2 = mpo2.get_full_matrix()
B = mpo1.get_product(mpo2).get_full_matrix()
assert torch.sum(torch.abs((A1 @ A2) - B)) < 1e-12